In [41]:

from northwellllm import NorthwellLLM
llm = NorthwellLLM(max_tokens=10000)
print(f"LLM max_tokens: {llm.max_tokens}")

LLM max_tokens: 10000


In [42]:
def fda_summary_to_es(document, model="o3", DEBUG=0):
    """
    Summarize an FDA 510(k) document in a JSON object for relevance assessment
    """
    lines = []
    lines.append('TASK: You are a skilled document analyst with expertise in regulatory laboratory affairs. your task is to efficiently analyze FDA 510(k) documents. Your primary goal is to distill these documents into succinct JSON summaries,that will help another language model judge the document\'s relevance to arbitrary search queries. You will be provided with an FDA 510(k) Summary document. Focus on extracting critical details necessary for swift relevancy assessment related to queries.')

    lines.append('EXTRACT THE FOLLOWING CRITICAL DETAILS:')
    lines.append('- Trade/Device Name(s): Array of trade/device names - extract all names for the submission mentioned in the document (e.g. [\"C.f.a.s. PUC\", \"Precinorm® PUC\", \"Precipath® PUC\"])')
    lines.append('- Submitter Information: Company name submitting the device (e.g. Roche Diagnostics)')
    lines.append('- 510(k) Number: Unique FDA submission identifier (e.g. K050026)')
    lines.append('- Predicate Device Reference 510(k) Number(s): Array of predicate devices with their K numbers (e.g. [\"K040264\", \"K041812\"])')
    lines.append('- Regulatory Class: FDA classification (e.g. Class II)')
    lines.append('- Product Code(s): Array of product codes (e.g. [\"DHE\", \"DHF\"]')
    lines.append('- Summary Letter Date')
    lines.append('- Summary Letter Received Date')
    lines.append('- Submission Date')
    lines.append('- Regulation Number(s) (e.g. 21CFR862.1150)')
    lines.append('- Regulation Name(s) (e.g. Cardiac Allograft Gene Expression Profiling Test System)')  
    lines.append('- Analyte Class(es): Array of any applicable classes. The only allowed classes are: chemistry, endocrine, allergy, autoimmune, toxicology, tdm, immunology, transplant, hematology, coagulation, microbiology, virology, mycology, mycobacteriology, parisitology, serology, flow cytometry, cytogenetics, genetic molecular, cancer molecular, anatomic pathology, cytology, POCT, misc, other. (e.g. [\"transplant\", \"genetic molecular\"])')
    lines.append('- Analyte(s): Array of specific analytes measured (e.g. [\"Albumin\", \"Total protein\", \"Immunoglobulin G\"])')
    lines.append('- Specimen Type(s): Array of specimen types (e.g. [\"Serum\", \"Plasma\", \"Whole Blood\", \"Cerebrospinal fluid\"])')
    lines.append('- Specimen Container(s): Array of specimen containers (e.g. [\"EDTA tube\", \"Heparin tube\"])')
    lines.append('- Instrument(s)/Platform(s): Array of instruments/platforms (e.g. [\"cobas® 6000 analyzer series\", \"cobas® 8000 modular analyzer series\"])')
    lines.append('- Method(s)/Technology(ies): Array of methods/technologies (e.g. [\"Enzyme immunoassay (EIA)\", \"Chemiluminescent immunoassay (CLIA)\"])')
    lines.append('- Methodologies: Array of methodologies (e.g. [\"Gene expression profiling\", \"RT-PCR\", \"Molecular expression testing\"])')
    lines.append('- Submission Type(s): Array of submission/device types. Laboratory terms common to all clinical assays should be omitted (e.g. [\"Test\", \"Control\"]). Common submission-specific types include: Test, Assay, Analyzer, Instrument, Kit, Calibrator, Control, Reference Material, Standard, System, Platform, Workstation, Monitoring System, Reagent, Software, Accessory, Component, Consumable, Panel, Multiplex, Point-of-care, Automated system, Specimen collection device, Sample preparation, Extraction kit, Detection system, Reader, Cartridge, Cassette, Strip test, Dipstick, Tube, Container, Transport medium, Diluent, Buffer, Wash solution, Substrate, Conjugate, Antibody, Primer, Probe, Quality control material, Proficiency testing material, Verification kit, Maintenance kit, Cleaning solution')
    lines.append('- Document Summary: one-line description of the document (e.g. FDA 510(k) summary for CareDx AlloMap gene expression profiling test using RT-PCR to monitor heart transplant rejection risk)')
    lines.append('- Indications for Use Summary: One-line summary of clinical use (e.g. "Aids in identifying heart transplant recipients (15+ years, 2+ months post-transplant) with stable allograft function who have low probability of moderate/severe acute cellular rejection, used in conjunction with standard clinical assessment.")')
    
    lines.append("""
    GUIDELINES:
    1) Extract all relevant information from the document to populate each field.
    2) Base everything strictly on the document text; do **not** invent facts or make up any information that is not present in the document.
    3) If a field is absent or unknown, output a blank field (or empty array for lists).
    4) Keep language neutral and factual; no personal opinions.
    5) Do not include this instruction set or any analysis of your reasoning in the output.
    6) Do not repeat tokens in keywords or entity names if they were previously listed in a field.
    7) It is crucial to match the JSON format below exactly and refrain from adding any additional fields or commentary.
    8) Ensure the output is valid JSON that can be parsed without errors.
    9) Any arrays should be output on one line, don't add a newline character after comma delimiters.
    """)

    lines.append('---------')
    lines.append('ORIGINAL FDA 510(k) Document')
    lines.append(document)
    lines.append('---------')
    lines.append('Return a JSON object structured as shown in the following example:')
    lines.append('''
    {
        "Trade/Device Name(s)": ["AlloMap Molecular Expression Testing", "Other Device Name"],
        "Submitter Information": "CareDx, Inc.",
        "510(k) Number": "K113081",
        "Predicate Device Reference 510(k) Number(s)": ["K040264", "K041812"],
        "Regulatory Class": "Class II",
        "Product Code(s)": ["MTO"],
        "Summary Letter Date": "March 15, 2012",
        "Summary Letter Received Date": "March 18, 2012", 
        "Submission Date": "November 30, 2011",
        "Regulation Number(s)": ["21CFR862.1150"],
        "Regulation Name(s)": ["Cardiac Allograft Gene Expression Profiling Test System"],
        "Analyte Class(es)": ["transplant", "genetic molecular"],
        "Analyte(s)": ["11 informative genes for cardiac rejection"],
        "Specimen Type(s)": ["Peripheral blood mononuclear cells"],
        "Specimen Container(s)": ["EDTA tube"],
        "Instrument(s)/Platform(s)": ["Applied Biosystems 7500 Real-Time PCR System"],
        "Method(s)/Technology(ies)": ["Real-time PCR", "Gene expression analysis"],
        "Methodologies": ["Gene expression profiling", "RT-PCR", "Molecular expression testing"],
        "Submission Type(s)": ["Test", "Assay"],
        "Document Summary": "FDA 510(k) summary for CareDx AlloMap gene expression profiling test using RT-PCR to monitor heart transplant rejection risk",
        "Indications for Use Summary": "Aids in identifying heart transplant recipients (15+ years, 2+ months post-transplant) with stable allograft function who have low probability of moderate/severe acute cellular rejection, used in conjunction with standard clinical assessment"
    }
    ''')
    lines.append('Please do not any wrapper text around the json object such as ```json ```. Only return the correctly formatted json object WITH ALL LISTS PRINTED ON THE SAME CONTINUOUS LINE.')
    
    prompt_text = ""
    for msg in lines:
        prompt_text += f"{'System'}: {msg}\n"
    from northwellllm import NorthwellLLM
    llm = NorthwellLLM(model=model, max_tokens=10000)
    print(f"LLM max_tokens: {llm.max_tokens}")
    llm_response = llm(prompt_text)
    if DEBUG>0:
        print(llm_response)
    if DEBUG>2:
        print(prompt_text)
    return llm_response



In [30]:
def es_to_file(es, fda_filename, save_folder, DEBUG=0):
    """
    Save the es to a file
    """
    import os
    es_folder = save_folder
    # Create es1 directory if it doesn't exist
    os.makedirs(es_folder, exist_ok=True)
    
    # Extract just the filename without path and replace extension
    base_filename = os.path.basename(fda_filename).replace('.md', '_es1.json')
    # print(base_filename)
    out_filename = os.path.join(es_folder, base_filename)
    print(out_filename)
    
    with open(out_filename, 'w') as f:
        f.write(es)
    if DEBUG>0:
        print(f"Saved es to {out_filename}")
    return out_filename

In [ ]:
# Main batch multithreaded processing script
import os
import glob
from concurrent.futures import ThreadPoolExecutor

## Process all FDA summary documents that DON'T have _REVIEW in the filename
all_fda_folders = '../LabDocs/fda_extracted_mds'
fda_summary_es1_folder = '../LabDocs/es1/fda_summary'

# iterate through all the fda folders, if a folder doesn't exist in the es1 folder create it
for fda_folder in os.listdir(all_fda_folders):
    fda_folder_path = os.path.join(all_fda_folders, fda_folder)
    print(f"PROCESSING FOLDER: {fda_folder}")
    # print the number of .md files in the fda_folder_path that don't have _REVIEW in the filename
    all_fda_docs = glob.glob(os.path.join(fda_folder_path, '*.md'))
    # print(f"Found {len(all_fda_docs)} FDA files in {fda_folder}")
    fda_summary_docs = [doc for doc in all_fda_docs if '_REVIEW' not in doc]
    print(f"Found {len(fda_summary_docs)} FDA summary files in {fda_folder}")

    out_es1_folder = os.path.join(fda_summary_es1_folder, os.path.basename(fda_folder))
    # print(f"Output folder: {out_es1_folder}")
    if not os.path.exists(out_es1_folder):
        print(f"Creating output folder: {out_es1_folder}")
        os.makedirs(out_es1_folder)

    def process_fda_doc(fda_summary_doc):
        #only process if the es1 file doesn't already exist
        base_filename = os.path.basename(fda_summary_doc).replace('.md', '_es1.json')
        out_filename = os.path.join(out_es1_folder, base_filename)
        if os.path.exists(out_filename):
            print(f"Skipping {fda_summary_doc}, es1 file already exists.")
            return out_filename
        with open(fda_summary_doc, 'r') as f:
            fda_document = f.read()
        es = fda_summary_to_es(fda_document, model="gpt-4.1", DEBUG=0)

        # add the fda_folder to the json object
        import json
        try:
            es_json = json.loads(es)
            es_json['fda_folder'] = fda_folder
            es = json.dumps(es_json, indent=4)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON for {fda_summary_doc}: {e}")
            return None

        es_file = es_to_file(es, fda_summary_doc, out_es1_folder, DEBUG=0)
        return es_file
    
    # check if the number of _es1.json files in the out_es1_folder is less than the number of .pdf fda_summary_docs (that don't have _REVIEW in the filename)
    es1_files = glob.glob(os.path.join(out_es1_folder, '*_es1.json'))
    if len(es1_files) < len(fda_summary_docs):
        print(f"**Processing {len(fda_summary_docs) - len(es1_files)} new documents in {fda_folder}")
        # if fda_folder == 'Cardiovascular':
        # skip to the first unprocessed document
        unprocessed_summary_docs = [doc for doc in fda_summary_docs if not os.path.exists(os.path.join(out_es1_folder, os.path.basename(doc).replace('.md', '_es1.json')))]
        print(f"Unprocessed documents: {len(unprocessed_summary_docs)}")
        if unprocessed_summary_docs:
            with ThreadPoolExecutor() as executor:
                results = list(executor.map(process_fda_doc, unprocessed_summary_docs))

    else:
        print(f"**All documents in {fda_folder} are already processed.")
    

PROCESSING FOLDER: Obstetrics
Found 5 FDA summary files in Obstetrics
**All documents in Obstetrics are already processed.
PROCESSING FOLDER: Toxicology
Found 518 FDA summary files in Toxicology
**All documents in Toxicology are already processed.
PROCESSING FOLDER: Hematology
Found 580 FDA summary files in Hematology
**All documents in Hematology are already processed.
PROCESSING FOLDER: Molecular Genetics
Found 2 FDA summary files in Molecular Genetics
**All documents in Molecular Genetics are already processed.
PROCESSING FOLDER: Clinical Chemistry
Found 1989 FDA summary files in Clinical Chemistry
**All documents in Clinical Chemistry are already processed.
PROCESSING FOLDER: Cardiovascular
Found 1 FDA summary files in Cardiovascular
**All documents in Cardiovascular are already processed.
PROCESSING FOLDER: Microbiology
Found 1114 FDA summary files in Microbiology
**All documents in Microbiology are already processed.
PROCESSING FOLDER: Immunology
Found 490 FDA summary files in Im

In [11]:
# Main batch multithreaded processing script
import os
import glob
from concurrent.futures import ThreadPoolExecutor

## Process all FDA summary documents that DON'T have _REVIEW in the filename
all_fda_folders = '../LabDocs/fda_extracted_mds'
fda_summary_es1_folder = '../LabDocs/es1/fda_summary'

# iterate through all the fda folders, if a folder doesn't exist in the es1 folder create it
for fda_folder in os.listdir(all_fda_folders):
    fda_folder_path = os.path.join(all_fda_folders, fda_folder)
    print(f"PROCESSING FOLDER: {fda_folder}")
    # print the number of .md files in the fda_folder_path that don't have _REVIEW in the filename
    all_fda_docs = glob.glob(os.path.join(fda_folder_path, '*.md'))
    # print(f"Found {len(all_fda_docs)} FDA files in {fda_folder}")
    fda_summary_docs = [doc for doc in all_fda_docs if '_REVIEW' not in doc]
    print(f"Found {len(fda_summary_docs)} FDA summary files in {fda_folder}")

    out_es1_folder = os.path.join(fda_summary_es1_folder, os.path.basename(fda_folder))
    # print(f"Output folder: {out_es1_folder}")
    if not os.path.exists(out_es1_folder):
        print(f"Creating output folder: {out_es1_folder}")
        os.makedirs(out_es1_folder)

    def process_fda_doc(fda_summary_doc):
        #only process if the es1 file doesn't already exist
        base_filename = os.path.basename(fda_summary_doc).replace('.md', '_es1.json')
        out_filename = os.path.join(out_es1_folder, base_filename)
        # if os.path.exists(out_filename):
        #     print(f"Skipping {fda_summary_doc}, es1 file already exists.")
        #     return out_filename
        with open(fda_summary_doc, 'r') as f:
            fda_document = f.read()
        es = fda_summary_to_es(fda_document, model="gpt-4.1", DEBUG=0)

        # add the fda_folder to the json object
        import json
        try:
            es_json = json.loads(es)
            es_json['fda_folder'] = fda_folder
            es = json.dumps(es_json, indent=4)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON for {fda_summary_doc}: {e}")
            return None

        es_file = es_to_file(es, fda_summary_doc, out_es1_folder, DEBUG=0)
        return es_file
    
    es1_files = glob.glob(os.path.join(out_es1_folder, '*_es1.json'))
    print(f"**Processing {len(fda_summary_docs) - len(es1_files)} new documents in {fda_folder}")
    #if fda_folder == 'Cardiovascular':
    with ThreadPoolExecutor() as executor:
        results = list(executor.map(process_fda_doc, fda_summary_docs))

    

PROCESSING FOLDER: Obstetrics
Found 5 FDA summary files in Obstetrics
**Processing 0 new documents in Obstetrics
../LabDocs/es1/fda_summary/Obstetrics/K151502_es1.json
../LabDocs/es1/fda_summary/Obstetrics/K103358_es1.json
../LabDocs/es1/fda_summary/Obstetrics/K160090_es1.json
../LabDocs/es1/fda_summary/Obstetrics/K150588_es1.json
../LabDocs/es1/fda_summary/Obstetrics/K153607_es1.json
PROCESSING FOLDER: Toxicology
Found 518 FDA summary files in Toxicology
**Processing 0 new documents in Toxicology
../LabDocs/es1/fda_summary/Toxicology/K163570_es1.json
../LabDocs/es1/fda_summary/Toxicology/K132174_es1.json
../LabDocs/es1/fda_summary/Toxicology/K080848_es1.json
../LabDocs/es1/fda_summary/Toxicology/K103656_es1.json
../LabDocs/es1/fda_summary/Toxicology/K170983_es1.json
../LabDocs/es1/fda_summary/Toxicology/K172910_es1.json
../LabDocs/es1/fda_summary/Toxicology/K121339_es1.json
../LabDocs/es1/fda_summary/Toxicology/K121166_es1.json
../LabDocs/es1/fda_summary/Toxicology/K050050_es1.json
..

In [45]:
import os

docs = [
"../LabDocs/fda_extracted_mds/Toxicology/K061655.md",
"../LabDocs/fda_extracted_mds/Clinical Chemistry/K062128.md",
"../LabDocs/fda_extracted_mds/Clinical Chemistry/K130162.md",
"../LabDocs/fda_extracted_mds/Clinical Chemistry/K193103.md",
"../LabDocs/fda_extracted_mds/Microbiology/K162950.md",
"../LabDocs/fda_extracted_mds/Microbiology/K142677.md",
"../LabDocs/fda_extracted_mds/Microbiology/K141133.md",
"../LabDocs/fda_extracted_mds/Microbiology/K163536.md",
"../LabDocs/fda_extracted_mds/Immunology/K124006.md"
]
fda_summary_es1_folder = '../LabDocs/es1/fda_summary'

for doc in docs:
    print(f"Processing {doc}")
    with open(doc, 'r') as f:
        fda_document = f.read()
    es = fda_summary_to_es(fda_document, model="gpt-4.1", DEBUG=1)

    # add the fda_folder to the json object
    import json
    try:
        es_json = json.loads(es)
        es_json['fda_folder'] = os.path.basename(os.path.dirname(doc))
        es = json.dumps(es_json, indent=4)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON for {doc}: {e} --- IGNORE ---")
        continue

    out_es1_folder = os.path.join(fda_summary_es1_folder, os.path.basename(os.path.dirname(doc)))
    print(f"Output folder: {out_es1_folder}")
    if not os.path.exists(out_es1_folder):
        # print(f"folder doesn't exist: {out_es1_folder}")
        print(f"Creating output folder: {out_es1_folder}")
        os.makedirs(out_es1_folder)

    es_file = es_to_file(es, doc, out_es1_folder, DEBUG=1)

Processing ../LabDocs/fda_extracted_mds/Toxicology/K061655.md
LLM max_tokens: 10000


{
    "Trade/Device Name(s)": ["Dimension Vista Acetaminophen (ACTM) Flex reagent cartridge", "Dimension Vista Amylase (AMY) Flex reagent cartridge", "Dimension Vista Creatine Kinase (CK) Flex reagent cartridge", "Dimension Vista Cholesterol (CHOL) Flex reagent cartridge", "Dimension Vista Gamma-glutamyl transferase (GGT) Flex reagent cartridge", "Dimension Vista Glucose (GLU) Flex reagent cartridge", "Dimension Vista High Density Lipoprotein Cholesterol (HDLC) Flex reagent cartridge", "Dimension Vista Low-Density Lipoprotein Cholesterol (LDLC) Flex reagent cartridge", "Dimension Vista Lidocaine (LIDO) Flex reagent cartridge", "Dimension Vista Magnesium (MG) Flex reagent cartridge", "Dimension Vista Pseudocholinesterase (PCHE) Flex reagent cartridge", "Dimension Vista Phosphorus (PHOS) Flex reagent cartridge", "Dimension Vista Procainamide (PROC) Flex reagent cartridge", "Dimension Vista Salicylate (SAL) Flex reagent cartridge", "Dimension Vista Thyroxine (T4) Flex reagent cartridge", 

In [ ]:
doc = "../LabDocs/fda_extracted_mds/Toxicology/K061655.md"
with open(doc, 'r') as f:
    fda_document = f.read()
es = fda_summary_to_es(fda_document, model="gpt-4.1", DEBUG=1)

# add the fda_folder to the json object
import json
try:
    es_json = json.loads(es)
    es_json['fda_folder'] = os.path.basename(os.path.dirname(doc))
    es = json.dumps(es_json, indent=4)
except json.JSONDecodeError as e:
    print(f"Error decoding JSON for {doc}: {e} --- IGNORE ---")

out_es1_folder = os.path.join(fda_summary_es1_folder, os.path.basename(os.path.dirname(doc)))
print(f"Output folder: {out_es1_folder}")
if not os.path.exists(out_es1_folder):
    # print(f"folder doesn't exist: {out_es1_folder}")
    print(f"Creating output folder: {out_es1_folder}")
    os.makedirs(out_es1_folder)

es_file = es_to_file(es, doc, out_es1_folder, DEBUG=1)

{
    "Trade/Device Name(s)": ["Dimension Vista Acetaminophen (ACTM) Flex reagent cartridge","Dimension Vista Amylase (AMY) Flex reagent cartridge","Dimension Vista Creatine Kinase (CK) Flex reagent cartridge","Dimension Vista Cholesterol (CHOL) Flex reagent cartridge","Dimension Vista Gamma-glutamyl transferase (GGT) Flex reagent cartridge","Dimension Vista Glucose (GLU) Flex reagent cartridge","Dimension Vista High Density Lipoprotein Cholesterol (HDLC) Flex reagent cartridge","Dimension Vista Low Density Lipoprotein Cholesterol (LDLC) Flex reagent cartridge","Dimension Vista Lidocaine (LIDO) Flex reagent cartridge","Dimension Vista Magnesium (MG) Flex reagent cartridge","Dimension Vista Pseudocholinesterase (PCHE) Flex reagent cartridge","Dimension Vista Phosphorus (PHOS) Flex reagent cartridge","Dimension Vista Procainamide (PROC) Flex reagent cartridge","Dimension Vista Salicylate (SAL) Flex reagent cartridge","Dimension Vista Thyroxine (T4) Flex reagent cartridge","Dimension Vist

In [43]:
doc = "../LabDocs/fda_extracted_mds/Toxicology/K061655.md"
with open(doc, 'r') as f:
    fda_document = f.read()
es = fda_summary_to_es(fda_document, model="gpt-4o", DEBUG=1)


LLM max_tokens: 10000


{
    "Trade/Device Name(s)": ["Dimension Vista™M Acetaminophen (ACTM) Flex® reagent cartridge", "Dimension Vista™M Amylase (AMY) Flex® reagent cartridge", "Dimension Vista™M Creatine Kinase (CK) Flex® reagent cartridge", "Dimension Vista™M Cholesterol (CHOL) Flex® reagent cartridge", "Dimension Vista™M Gamma-glutamyl transferase (GGT) Flex® reagent cartridge", "Dimension Vista™M Glucose (GLU) Flex® reagent cartridge", "Dimension Vista™M High Density Lipoprotein Cholesterol (HDLC) Flex® reagent cartridge", "Dimension Vista™M Low Density Lipoprotein Cholesterol (LDLC) Flex® reagent cartridge", "Dimension Vista™M Lidocaine (LIDO) Flex® reagent cartridge", "Dimension Vista™M Magnesium (MG) Flex® reagent cartridge", "Dimension Vista™M Pseudocholinesterase (PCHE) Flex® reagent cartridge", "Dimension Vista™M Phosphorus (PHOS) Flex® reagent cartridge", "Dimension Vista™M Procainamide (PROC) Flex® reagent cartridge", "Dimension Vista™M Salicylate (SAL) Flex® reagent cartridge", "Dimension Vist

In [35]:
import os
print(os.getcwd())

/labs/jupyter/ai_challenge/prod/preprocess


In [86]:
# fda_doc_path = '../LabDocs/fda_summary_extracted_txts/K221640.md'
fda_doc_path = '../LabDocs/fda_extracted_mds/Clinical Chemistry/K050026.md'

out_es1_folder = '../LabDocs/es1/fda_summary/Cardiovascular'

with open(fda_doc_path, 'r') as f:
    fda_document = f.read()
es = fda_summary_to_es(fda_document, model="gpt-4o", DEBUG=1)
es_file = es_to_file(es, fda_doc_path, out_es1_folder, DEBUG=0)  

{
    "Trade/Device Name(s)": ["C.f.a.s. PUC", "Precinorm® PUC", "Precipath® PUC"],
    "Submitter Information": "Roche Diagnostics",
    "510(k) Number": "K050026",
    "Predicate Device Reference 510(k) Number(s)": ["K040264", "K041812"],
    "Regulatory Class": "Class II",
    "Product Code(s)": ["JIX", "JJY"],
    "Summary Letter Date": "",
    "Summary Letter Received Date": "January 6, 2005",
    "Submission Date": "January 5, 2005",
    "Regulation Number(s)": ["21 CFR 862.1150"],
    "Regulation Name(s)": ["Calibrator"],
    "Analyte Class(es)": ["chemistry"],
    "Analyte(s)": ["Albumin", "Total protein", "Immunoglobulin G", "Creatinine", "Immunoglobulin A", "Immunoglobulin M"],
    "Specimen Type(s)": ["Urine", "CSF"],
    "Specimen Container(s)": [],
    "Instrument(s)/Platform(s)": ["Roche clinical chemistry analyzers"],
    "Method(s)/Technology(ies)": ["Calibration", "Quality control"],
    "Methodologies": [],
    "Submission Type(s)": ["Calibrator", "Control"],
    "Doc

In [43]:
fda_doc_path = '../LabDocs/fda_summary_extracted_txts/K221640.md'
out_es1_folder = '../LabDocs/es1/fda_summary/Cardiovascular'

with open(fda_doc_path, 'r') as f:
    fda_document = f.read()
es = fda_summary_to_es(fda_document, model="o3", DEBUG=1)
es_file = es_to_file(es, fda_doc_path, out_es1_folder, DEBUG=0) 



{
    "Trade/Device Name(s)": ["AlloMap Heart Molecular Expression Testing", "AlloMap Heart Testing"],
    "Submitter Information": "CareDx, Inc.",
    "510(k) Number": "K221640",
    "Predicate Device Reference 510(k) Number(s)": ["K073482"],
    "Regulatory Class": "Class II",
    "Product Code(s)": ["OJQ"],
    "Summary Letter Date": "September 13, 2023",
    "Summary Letter Received Date": "April 10, 2023",
    "Submission Date": "April 7, 2023",
    "Regulation Number(s)": ["21 CFR 862.1163"],
    "Regulation Name(s)": ["Cardiac Allograft Gene Expression Profiling Test System"],
    "Analyte Class(es)": ["transplant", "genetic molecular"],
    "Analyte(s)": ["11 informative genes associated with cardiac rejection"],
    "Specimen Type(s)": ["Peripheral blood mononuclear cells"],
    "Specimen Container(s)": ["BD Vacutainer CPT Cell Preparation tube (sodium citrate)"],
    "Instrument(s)/Platform(s)": ["Roche LightCycler 480 II", "Applied Biosystems 7900HT Thermal Cycler"],
    "Me

In [84]:
# fda_doc_path = '../LabDocs/fda_summary_extracted_txts/K221640.md'
fda_doc_path = '../LabDocs/fda_extracted_mds/Clinical Chemistry/K050026.md'

out_es1_folder = '../LabDocs/es1/fda_summary/Cardiovascular'

with open(fda_doc_path, 'r') as f:
    fda_document = f.read()
es = fda_summary_to_es(fda_document, model="gpt-4.1", DEBUG=1)
es_file = es_to_file(es, fda_doc_path, out_es1_folder, DEBUG=0)  

{
    "Trade/Device Name(s)": ["C.f.a.s. PUC", "Precinorm® PUC", "Precipath® PUC"],
    "Submitter Information": "Roche Diagnostics",
    "510(k) Number": "K050026",
    "Predicate Device Reference 510(k) Number(s)": ["K040264", "K041812"],
    "Regulatory Class": "Class II",
    "Product Code(s)": ["JIX", "JJY"],
    "Summary Letter Date": "January 31, 2005",
    "Summary Letter Received Date": "January 6, 2005",
    "Submission Date": "January 5, 2005",
    "Regulation Number(s)": ["21CFR862.1150"],
    "Regulation Name(s)": ["Calibrator"],
    "Analyte Class(es)": ["chemistry"],
    "Analyte(s)": ["Albumin", "Total protein", "Immunoglobulin G", "Creatinine", "Immunoglobulin A", "Immunoglobulin M"],
    "Specimen Type(s)": ["Urine", "Cerebrospinal fluid"],
    "Specimen Container(s)": [],
    "Instrument(s)/Platform(s)": ["Roche clinical chemistry analyzers"],
    "Method(s)/Technology(ies)": ["Quantitative calibration", "Quality control"],
    "Methodologies": [],
    "Submission Ty

In [85]:
# fda_doc_path = '../LabDocs/fda_summary_extracted_txts/K221640.md'
fda_doc_path = '../LabDocs/fda_extracted_mds/Clinical Chemistry/K050026.md'

out_es1_folder = '../LabDocs/es1/fda_summary/Cardiovascular'

with open(fda_doc_path, 'r') as f:
    fda_document = f.read()
es = fda_summary_to_es(fda_document, model="gpt-4.1-nano", DEBUG=1)
es_file = es_to_file(es, fda_doc_path, out_es1_folder, DEBUG=0)  

{
  "Trade/Device Name(s)": [
    "C.f.a.s. PUC",
    "Precinorm® PUC",
    "Precipath® PUC"
  ],
  "Submitter Information": "Roche Diagnostics",
  "510(k) Number": "K050026",
  "Predicate Device Reference 510(k) Number(s)": [
    "K040264",
    "K041812"
  ],
  "Regulatory Class": "Class II",
  "Product Code(s)": [
    "JIX",
    "JJY"
  ],
  "Summary Letter Date": "January 5, 2005",
  "Summary Letter Received Date": "January 6, 2005",
  "Submission Date": "January 5, 2005",
  "Regulation Number(s)": [
    "21CFR862.1150"
  ],
  "Regulation Name(s)": [
    "Calibrator"
  ],
  "Analyte Class(es)": [],
  "Analyte(s)": [],
  "Specimen Type(s)": [],
  "Specimen Container(s)": [],
  "Instrument(s)/Platform(s)": [],
  "Method(s)/Technology(ies)": [],
  "Methodologies": [],
  "Submission Type(s)": [],
  "Document Summary": "FDA 510(k) summary for calibrators and controls including C.f.a.s. PUC, Precinorm® PUC, and Precipath® PUC for clinical chemistry calibration and quality control",
  "Ind

In [80]:
# fda_doc_path = '../LabDocs/fda_summary_extracted_txts/K221640.md'
fda_doc_path = '../LabDocs/fda_extracted_mds/Clinical Chemistry/K050026.md'

out_es1_folder = '../LabDocs/es1/fda_summary/Cardiovascular'

with open(fda_doc_path, 'r') as f:
    fda_document = f.read()
es = fda_summary_to_es(fda_document, model="gpt-4.1-mini", DEBUG=1)
es_file = es_to_file(es, fda_doc_path, out_es1_folder, DEBUG=0)  

{
    "Trade/Device Name(s)": [
        "C.f.a.s. (Calibrator for Automated Systems) PUC (Proteins in Urine/CSF)",
        "C.f.a.s. PUC",
        "Precinorm® PUC (Proteins in Urine/CSF)",
        "Precipath® PUC (Proteins in Urine/CSF)"
    ],
    "Submitter Information": "Roche Diagnostics",
    "510(k) Number": "K050026",
    "Predicate Device Reference 510(k) Number(s)": [
        "K040264",
        "K041812"
    ],
    "Regulatory Class": "Class II",
    "Product Code(s)": [
        "JIX",
        "JJY"
    ],
    "Summary Letter Date": "January 5, 2005",
    "Summary Letter Received Date": "January 6, 2005",
    "Submission Date": "January 5, 2005",
    "Regulation Number(s)": [
        "21 CFR 862.1150"
    ],
    "Regulation Name(s)": [
        "Calibrator"
    ],
    "Analyte Class(es)": [
        "chemistry"
    ],
    "Analyte(s)": [
        "Albumin",
        "Total protein",
        "Immunoglobulin G",
        "Creatinine",
        "Immunoglobulin A",
        "Immunoglobul

In [76]:
# fda_doc_path = '../LabDocs/fda_summary_extracted_txts/K221640.md'
fda_doc_path = '../LabDocs/fda_extracted_mds/Clinical Chemistry/K050026.md'

out_es1_folder = '../LabDocs/es1/fda_summary/Cardiovascular'

with open(fda_doc_path, 'r') as f:
    fda_document = f.read()
es = fda_summary_to_es(fda_document, model="gpt-4o-mini", DEBUG=1)
es_file = es_to_file(es, fda_doc_path, out_es1_folder, DEBUG=0)  

{
    "Trade/Device Name(s)": ["C.f.a.s. PUC", "Precinorm® PUC", "Precipath® PUC"],
    "Submitter Information": "Roche Diagnostics",
    "510(k) Number": "K050026",
    "Predicate Device Reference 510(k) Number(s)": ["K040264", "K041812"],
    "Regulatory Class": "Class II",
    "Product Code(s)": ["JIX", "JJY"],
    "Summary Letter Date": "January 31, 2005",
    "Summary Letter Received Date": "January 6, 2005",
    "Submission Date": "January 5, 2005",
    "Regulation Number(s)": ["21 CFR 862.1150"],
    "Regulation Name(s)": ["Calibrator"],
    "Analyte Class(es)": [],
    "Analyte(s)": ["Albumin", "Total protein", "Immunoglobulin G", "Creatinine", "Immunoglobulin A", "Immunoglobulin M"],
    "Specimen Type(s)": ["Urine", "Cerebrospinal fluid"],
    "Specimen Container(s)": [],
    "Instrument(s)/Platform(s)": ["Roche clinical chemistry analyzers"],
    "Method(s)/Technology(ies)": [],
    "Methodologies": [],
    "Submission Type(s)": ["Calibrator", "Control"],
    "Document Summa

In [78]:
# fda_doc_path = '../LabDocs/fda_summary_extracted_txts/K221640.md'
fda_doc_path = '../LabDocs/fda_extracted_mds/Clinical Chemistry/K050026.md'

out_es1_folder = '../LabDocs/es1/fda_summary/Cardiovascular'

with open(fda_doc_path, 'r') as f:
    fda_document = f.read()
es = fda_summary_to_es(fda_document, model="gpt-4o", DEBUG=1)
es_file = es_to_file(es, fda_doc_path, out_es1_folder, DEBUG=0)  

{
    "Trade/Device Name(s)": ["C.f.a.s. PUC", "Precinorm® PUC", "Precipath® PUC"],
    "Submitter Information": "Roche Diagnostics",
    "510(k) Number": "K050026",
    "Predicate Device Reference 510(k) Number(s)": ["K040264", "K041812"],
    "Regulatory Class": "Class II",
    "Product Code(s)": ["JIX", "JJY"],
    "Summary Letter Date": "",
    "Summary Letter Received Date": "",
    "Submission Date": "January 5, 2005",
    "Regulation Number(s)": ["21 CFR 862.1150"],
    "Regulation Name(s)": ["Calibrator"],
    "Analyte Class(es)": [],
    "Analyte(s)": ["Albumin", "Total protein", "Immunoglobulin G", "Creatinine", "Immunoglobulin A", "Immunoglobulin M"],
    "Specimen Type(s)": [],
    "Specimen Container(s)": [],
    "Instrument(s)/Platform(s)": ["Roche clinical chemistry analyzers"],
    "Method(s)/Technology(ies)": [],
    "Methodologies": [],
    "Submission Type(s)": ["Calibrator", "Control"],
    "Document Summary": "FDA 510(k) summary for Roche Diagnostics C.f.a.s. PUC, 

In [58]:
fda_doc_path = '../LabDocs/fda_summary_extracted_txts/K221640.md'
out_es1_folder = '../LabDocs/es1/fda_summary/Cardiovascular'

with open(fda_doc_path, 'r') as f:
    fda_document = f.read()
es = fda_summary_to_es(fda_document, model="claude-sonnet-4", DEBUG=1)
es_file = es_to_file(es, fda_doc_path, out_es1_folder, DEBUG=0)  

{
    "Trade/Device Name(s)": ["AlloMap Heart Molecular Expression Testing"],
    "Submitter Information": "CareDx, Inc.",
    "510(k) Number": "K221640",
    "Predicate Device Reference 510(k) Number(s)": ["K073482"],
    "Regulatory Class": "Class II",
    "Product Code(s)": ["OJQ"],
    "Summary Letter Date": "September 13, 2023",
    "Summary Letter Received Date": "April 10, 2023",
    "Submission Date": "April 7, 2023",
    "Regulation Number(s)": ["21CFR862.1163"],
    "Regulation Name(s)": ["Cardiac Allograft Gene Expression Profiling Test System"],
    "Analyte Class(es)": ["transplant", "genetic molecular"],
    "Analyte(s)": ["11 informative genes", "9 control genes"],
    "Specimen Type(s)": ["Peripheral blood mononuclear cells", "Whole blood"],
    "Specimen Container(s)": ["BD Vacutainer CPT Cell Preparation tube with sodium citrate anticoagulant"],
    "Instrument(s)/Platform(s)": ["Roche LightCycler 480 II"],
    "Method(s)/Technology(ies)": ["qRT-PCR", "Gene expression

In [60]:
fda_doc_path = '../LabDocs/fda_summary_extracted_txts/K221640.md'
out_es1_folder = '../LabDocs/es1/fda_summary/Cardiovascular'

with open(fda_doc_path, 'r') as f:
    fda_document = f.read()
es = fda_summary_to_es(fda_document, model="claude-3.5-sonnet", DEBUG=1)
es_file = es_to_file(es, fda_doc_path, out_es1_folder, DEBUG=0)  

{
    "Trade/Device Name(s)": ["AlloMap Heart Molecular Expression Testing", "AlloMap Heart Testing"],
    "Submitter Information": "CareDx, Inc.",
    "510(k) Number": "K221640",
    "Predicate Device Reference 510(k) Number(s)": ["K073482"],
    "Regulatory Class": "Class II",
    "Product Code(s)": ["OJQ"],
    "Summary Letter Date": "September 13, 2023",
    "Summary Letter Received Date": "April 10, 2023",
    "Submission Date": "April 7, 2023",
    "Regulation Number(s)": ["21 CFR 862.1163"],
    "Regulation Name(s)": ["Cardiac Allograft Gene Expression Profiling Test System"],
    "Analyte Class(es)": ["transplant", "genetic molecular"],
    "Analyte(s)": ["RNA from peripheral blood mononuclear cells"],
    "Specimen Type(s)": ["Peripheral blood mononuclear cells"],
    "Specimen Container(s)": ["BD Vacutainer CPT Cell Preparation tube with sodium citrate anticoagulant"],
    "Instrument(s)/Platform(s)": ["Roche LightCycler 480 II"],
    "Method(s)/Technology(ies)": ["Real-time 

In [61]:
fda_doc_path = '../LabDocs/fda_summary_extracted_txts/K221640.md'
out_es1_folder = '../LabDocs/es1/fda_summary/Cardiovascular'

with open(fda_doc_path, 'r') as f:
    fda_document = f.read()
es = fda_summary_to_es(fda_document, model="claude-3.5-haiku", DEBUG=1)
es_file = es_to_file(es, fda_doc_path, out_es1_folder, DEBUG=0)  

{
    "Trade/Device Name(s)": ["AlloMap Heart Molecular Expression Testing", "AlloMap Heart Testing"],
    "Submitter Information": "CareDx, Inc.",
    "510(k) Number": "K221640",
    "Predicate Device Reference 510(k) Number(s)": ["K073482"],
    "Regulatory Class": "Class II",
    "Product Code(s)": ["OJQ"],
    "Summary Letter Date": "April 7, 2023",
    "Summary Letter Received Date": "April 10, 2023",
    "Submission Date": null,
    "Regulation Number(s)": ["21 CFR 862.1163"],
    "Regulation Name(s)": ["Cardiac Allograft Gene Expression Profiling Test System"],
    "Analyte Class(es)": ["transplant", "genetic molecular"],
    "Analyte(s)": ["20 gene panel (11 informative, 9 control genes)"],
    "Specimen Type(s)": ["Peripheral blood mononuclear cells (PBMC)"],
    "Specimen Container(s)": ["BD Vacutainer CPT™ Cell Preparation tube"],
    "Instrument(s)/Platform(s)": ["Roche LightCycler 480 II (LC480)"],
    "Method(s)/Technology(ies)": ["Quantitative RT-PCR"],
    "Methodologie

In [70]:
fda_doc_path = '../LabDocs/fda_summary_extracted_txts/K221640.md'
out_es1_folder = '../LabDocs/es1/fda_summary/Cardiovascular'

with open(fda_doc_path, 'r') as f:
    fda_document = f.read()
es = fda_summary_to_es(fda_document, model="gemini-2.0-flash", DEBUG=1)
es_file = es_to_file(es, fda_doc_path, out_es1_folder, DEBUG=0)  

```json
{
    "Trade/Device Name(s)": ["AlloMap Heart Molecular Expression Testing", "AlloMap Heart Testing"],
    "Submitter Information": "CareDx, Inc.",
    "510(k) Number": "K221640",
    "Predicate Device Reference 510(k) Number(s)": ["K073482"],
    "Regulatory Class": "Class II",
    "Product Code(s)": ["OJQ"],
    "Summary Letter Date": "September 13, 2023",
    "Summary Letter Received Date": "April 10, 2023",
    "Submission Date": "September 6, 2023",
    "Regulation Number(s)": ["21 CFR 862.1163"],
    "Regulation Name(s)": ["Cardiac Allograft Gene Expression Profiling Test System"],
    "Analyte Class(es)": ["transplant", "genetic molecular"],
    "Analyte(s)": ["20 gene assays", "11 informative genes"],
    "Specimen Type(s)": ["peripheral blood mononuclear cells", "PBMC", "Whole blood"],
    "Specimen Container(s)": ["BD Vacutainer CPT Cell Preparation tube with sodium citrate anticoagulant"],
    "Instrument(s)/Platform(s)": ["ABI 7900HT thermal cyclers", "Roche LightCy

In [64]:
fda_doc_path = '../LabDocs/fda_summary_extracted_txts/K221640.md'
out_es1_folder = '../LabDocs/es1/fda_summary/Cardiovascular'

with open(fda_doc_path, 'r') as f:
    fda_document = f.read()
es = fda_summary_to_es(fda_document, model="gemini-2.0-flash-lite", DEBUG=1)
es_file = es_to_file(es, fda_doc_path, out_es1_folder, DEBUG=0)  

```json
{
  "Trade/Device Name(s)": [
    "AlloMap Heart Molecular Expression Testing",
    "AlloMap Heart Testing"
  ],
  "Submitter Information": "CareDx, Inc.",
  "510(k) Number": "K221640",
  "Predicate Device Reference 510(k) Number(s)": [
    "K073482"
  ],
  "Regulatory Class": "Class II",
  "Product Code(s)": [
    "OJQ"
  ],
  "Summary Letter Date": "September 13, 2023",
  "Summary Letter Received Date": "April 10, 2023",
  "Submission Date": "April 7, 2023",
  "Regulation Number(s)": [
    "21 CFR 862.1163"
  ],
  "Regulation Name(s)": [
    "Cardiac Allograft Gene Expression Profiling Test System"
  ],
  "Analyte Class(es)": [
    "transplant",
    "genetic molecular"
  ],
  "Analyte(s)": [
    "RNA"
  ],
  "Specimen Type(s)": [
    "Peripheral blood mononuclear cells"
  ],
  "Specimen Container(s)": [
    "BD Vacutainer CPT™ Cell Preparation tube with sodium citrate anticoagulant"
  ],
  "Instrument(s)/Platform(s)": [
    "Roche LightCycler 480 II (LC480)"
  ],
  "Method(s)

In [65]:
fda_doc_path = '../LabDocs/fda_summary_extracted_txts/K221640.md'
out_es1_folder = '../LabDocs/es1/fda_summary/Cardiovascular'

with open(fda_doc_path, 'r') as f:
    fda_document = f.read()
es = fda_summary_to_es(fda_document, model="gemini-1.5-flash", DEBUG=1)
es_file = es_to_file(es, fda_doc_path, out_es1_folder, DEBUG=0)  

```json
{
  "Trade/Device Name(s)": [
    "AlloMap Heart Molecular Expression Testing",
    "AlloMap Heart Testing",
    "AlloMap Molecular Expression Testing",
    "AlloMap Testing"
  ],
  "Submitter Information": "CareDx, Inc.",
  "510(k) Number": "K221640",
  "Predicate Device Reference 510(k) Number(s)": [
    "K073482"
  ],
  "Regulatory Class": "Class II",
  "Product Code(s)": [
    "OJQ"
  ],
  "Summary Letter Date": "April 7, 2023",
  "Summary Letter Received Date": "April 10, 2023",
  "Submission Date": null,
  "Regulation Number(s)": [
    "21 CFR 862.1163"
  ],
  "Regulation Name(s)": [
    "Cardiac Allograft Gene Expression Profiling Test System"
  ],
  "Analyte Class(es)": [
    "transplant",
    "genetic molecular"
  ],
  "Analyte(s)": [
    "20 genes (11 informative and 9 control genes)"
  ],
  "Specimen Type(s)": [
    "Peripheral blood mononuclear cells (PBMC)",
    "Whole blood"
  ],
  "Specimen Container(s)": [
    "BD Vacutainer CPT™ Cell Preparation tube with sodiu

In [68]:
fda_doc_path = '../LabDocs/fda_summary_extracted_txts/K221640.md'
out_es1_folder = '../LabDocs/es1/fda_summary/Cardiovascular'

with open(fda_doc_path, 'r') as f:
    fda_document = f.read()
es = fda_summary_to_es(fda_document, model="gemini-2.5-pro", DEBUG=1)
es_file = es_to_file(es, fda_doc_path, out_es1_folder, DEBUG=0)  

```json
{
    "Trade/Device Name(s)": [
        "AlloMap Heart Molecular Expression Testing",
        "AlloMap Heart Testing"
    ],

K221640_es1.json
../LabDocs/es1/fda_summary/Cardiovascular/K221640_es1.json


In [44]:
fda_doc_path = '../LabDocs/fda_summary_extracted_txts/K221640.md'
out_es1_folder = '../LabDocs/es1/fda_summary/Cardiovascular'

with open(fda_doc_path, 'r') as f:
    fda_document = f.read()
es = fda_summary_to_es(fda_document, model="o4-mini", DEBUG=1)
es_file = es_to_file(es, fda_doc_path, out_es1_folder, DEBUG=0)  

{
    "Trade/Device Name(s)": [
        "AlloMap Heart Molecular Expression Testing",
        "AlloMap Heart Testing"
    ],
    "Submitter Information": "CareDx, Inc.",
    "510(k) Number": "K221640",
    "Predicate Device Reference 510(k) Number(s)": [
        "K073482"
    ],
    "Regulatory Class": "Class II",
    "Product Code(s)": [
        "OJQ"
    ],
    "Summary Letter Date": "September 13, 2023",
    "Summary Letter Received Date": "April 10, 2023",
    "Submission Date": "April 7, 2023",
    "Regulation Number(s)": [
        "21 CFR 862.1163"
    ],
    "Regulation Name(s)": [
        "Cardiac Allograft Gene Expression Profiling Test System"
    ],
    "Analyte Class(es)": [
        "transplant",
        "genetic molecular"
    ],
    "Analyte(s)": [
        "Expression of 11 informative genes",
        "Expression of 9 control genes"
    ],
    "Specimen Type(s)": [
        "Whole blood",
        "Peripheral blood mononuclear cells"
    ],
    "Specimen Container(s)": [
  

In [ ]:
import os
# fda_doc_path = '../LabDocs/fda_summary_extracted_txts/K050026.txt'
# fda_doc_path = '../LabDocs/fda_summary_extracted_txts/K050037.txt'
# fda_doc_path = '../LabDocs/fda_summary_extracted_txts/K221640.txt'
# fda_doc_path = '../LabDocs/fda_summary_extracted_txts/K050050.txt'

# fda_doc_path = '../LabDocs/fda_summary_extracted_txts/K050026.md'
# fda_doc_path = '../LabDocs/fda_summary_extracted_txts/K050037.md'
fda_doc_path = '../LabDocs/fda_summary_extracted_txts/K221640.md'
# fda_doc_path = '../LabDocs/fda_summary_extracted_txts/K050050.md'

with open(fda_doc_path, 'r') as f:
    fda_doc = f.read()
es = fda_summary_to_es(fda_doc, model="gpt-4o", DEBUG=1)
# es = fda_summary_to_es(fda_doc, model="o3", DEBUG=2)


fda_filename_with_extension = os.path.basename(fda_doc_path)
# fda_filename, _ = os.path.splitext(fda_filename_with_extension)
print(fda_filename_with_extension)
# es_filename = os.path.join("../LabDocs/es1/fda", fda_filename + ".json")

# print(es_filename)


es_to_file(es, fda_filename_with_extension, DEBUG=1)

FileNotFoundError: [Errno 2] No such file or directory: '../LabDocs/fda_summary_extracted_txts/K050050.md'